In [128]:
#!pip install tensorflow
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import preprocess_input
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn import mixture
from scipy.misc import imresize
import io, os, sys, types, shutil, glob, os.path
from pathlib import Path
import numpy as np
import pickle
import random
import math

In [120]:
image.LOAD_TRUNCATED_IMAGES = True 
model = VGG16(weights='imagenet', include_top=False)
res_model = ResNet50(weights='imagenet', include_top=False)
mobile_model = MobileNet(weights='imagenet', include_top=False)

17227776/17225924 [==============================] - 4s 0us/step


In [146]:
def run_kmeans(images):
    cwd = os.getcwd() 

    feature_list = []
    #pict_dict = dict(zip(pict_list, range(0, len(pict_list))))
    #print(pict_dict)
    pict_list = images
    random.shuffle(pict_list)
    image.smart_resize(
        pict_list, (224, 224), interpolation='bilinear')

    for i, imagepath in enumerate(pict_list):
        print("    Status: %s / %s" %(i, len(pict_list)), end="\r")
        #img = image.load_img(imagepath, target_size=(224, 224))
        #img = imresize(imagepath, (224,224))
        img_data = image.img_to_array(imagepath)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        features = np.array(mobile_model.predict(img_data))
        feature_list.append(features.flatten())


    sil = []
    kmax = 6
    x = np.array(feature_list)

    for k in range(2, kmax+1):
        km = KMeans(n_clusters = k).fit(x)
        labels = km.labels_
        sil.append(silhouette_score(x, labels, metric = 'euclidean'))

    scores = [abs(sil[i] - sil[i-1]) for i in range(1, len(sil))]
    k = scores.index(min(scores)) + 2

    km = KMeans(n_clusters = k).fit(x)
    dists = np.array(km.transform(x)).T
    ind = [i.argmin() for i in dists]
    
    
    targetdir = (cwd + "\\target_dir\\")
    procdir = (cwd + "\\proc_dir\\")
    try:
        if os.path.exists(targetdir):
            shutil.rmtree(targetdir)
        if os.path.exists(procdir):
            shutil.rmtree(targetdir)
        os.makedirs(targetdir)
        os.makedirs(procdir)
    except OSError:
        pass

    # Copy with cluster name
    print("\n")
    for i, m in enumerate(km.labels_):
        print("    Copy: %s / %s" %(i, len(km.labels_)), end="\r")
        shutil.copy(pict_list[i], targetdir + str(m) + "_" + str(i) + ".jpg")

    for i in ind:
        shutil.copy(pict_list[i], procdir + str(km.labels_[i]) + "_" + str(i) + ".jpg")
        
    return [pict_list[i] for i in ind]

[[150.93416 150.93416 378.1541  384.69812 355.44345 387.5734  367.6183
  377.6572  376.13187 376.29648]
 [349.36673 357.9215  185.92755 371.29572 177.96031 358.05884 182.49141
  369.1195  192.90305 345.36072]
 [372.00357 373.58997 358.58234 374.7762  354.39426 172.36342 357.7298
  372.39847 361.56625 172.36342]
 [365.73492 372.64258 368.31372 178.2629  373.76837 374.0919  372.75452
  178.2629  378.89136 367.49976]] [0, 4, 5, 3]
0 0
1 0
2 1
3 3
4 1
5 2
6 1
7 3
8 1
9 2
